In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
cd /content/MyDrive/My Drive/Colab Notebooks/mask-detection-master

In [ ]:
#Data Set을 만들어서 학습.
from google.colab.patches import cv2_imshow

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

facenet = cv2.dnn.readNet('models/deploy.prototxt', 'models/res10_300x300_ssd_iter_140000.caffemodel')
#FaceDetector 모델 > OpenCv의 DNN
model = load_model('models/mask_detector.model')
#MaskDetector 모델 > Keras 모델

cap = cv2.VideoCapture('imgs/01.mp4')
#동영상 로드
ret, img = cap.read()

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (img.shape[1], img.shape[0]))

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    h, w = img.shape[:2]

    #Preprocessing. OpenCV의 FaceNet에서 학습시킨대로 Param값을 넣어줌. DNN이 사용하는 형태로 이미지 변환
    blob = cv2.dnn.blobFromImage(img, scalefactor=1., size=(300, 300), mean=(104., 177., 123.))
    facenet.setInput(blob) #변환해준 이미지 FaceNet의 input
    dets = facenet.forward() #facedection 결과 저장

    result_img = img.copy()

    for i in range(dets.shape[2]): #저장이 된 것을 loop을 돌면서 저장
        confidence = dets[0, 0, i, 2] #검사하는데 detection의 결과가 자신있는 정도.
        if confidence < 0.5:
            continue

        x1 = int(dets[0, 0, i, 3] * w) #bounding 박스 구해주기
        y1 = int(dets[0, 0, i, 4] * h)
        x2 = int(dets[0, 0, i, 5] * w)
        y2 = int(dets[0, 0, i, 6] * h)
        
        face = img[y1:y2, x1:x2] #bounding Box을 통해 얼굴만 저장

        #마스크를 썼나 안썼나 예측
        # 전처리하는 부분
        face_input = cv2.resize(face, dsize=(224, 224)) #이미지 크기 변경
        face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB) #이미지의 컬러시스템 변경
        face_input = preprocess_input(face_input) #mobileNetV2에서 하는 preprocessing과 똑같이 하기위해 처리
        face_input = np.expand_dims(face_input, axis=0)  #이렇게 하면 shape이 (224,224,3) 으로 나오는데 넣을때는 (1,224,224,3)이 되어야 하므로 차원하나 추가
        
        mask, nomask = model.predict(face_input).squeeze() #load해놓은 모델에 predict method를 통해, 마스크 여부 확률을 반환

        if mask > nomask:
            color = (0, 255, 0)
            label = 'Mask %d%%' % (mask * 100)
        else:
            color = (0, 0, 255)
            label = 'No Mask %d%%' % (nomask * 100)

        cv2.rectangle(result_img, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
        cv2.putText(result_img, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=color, thickness=2, lineType=cv2.LINE_AA)

    out.write(result_img)
    cv2_imshow(result_img)
    #cv2.imshow('result', result_img)
    if cv2.waitKey(1) == ord('q'):
        break

out.release()
cap.release()